In [1]:
import numpy as np
import pandas as pd
import os
import re
import tensorflow as tf
from datetime import datetime
from tensorflow import keras
from pandas.plotting import table
import matplotlib.pyplot as plt
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam
from DataLoader import Predictions_Input,map_CU,generate_Depth

/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorf

In [2]:
Depth_0 = keras.models.load_model('Model_Depth0_adam_63.h5')
Depth_1 = keras.models.load_model('Model_Depth1_adam_82.h5')
Depth_2 = keras.models.load_model('Model_Depth2_adam_91.h5')
os.chdir("Videos/Dados DeepLearning")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
os.chdir("../Videos Validacao/Dados Deep")
os.getcwd()

'/home/tomas/Desktop/TCC/Deeplearning_Video_Compressing/Videos/Videos Validacao/Dados Deep'

In [4]:
Lista = os.listdir("./")
Lista = list(filter(lambda x : "." not in x,Lista))

In [6]:
Lista = list(filter(lambda x : "Kimon" in x or "ParkS" in x,Lista))

In [7]:
Lista.sort()
Lista

['Kimon_22',
 'Kimon_27',
 'Kimon_32',
 'Kimon_37',
 'ParkS_22',
 'ParkS_27',
 'ParkS_32',
 'ParkS_37']

In [8]:
for folder in Lista:
    Dados = Predictions_Input(folder)
    Input,Video = Dados.generate_input()
    string_2 = ""
    entra = datetime.now()
    print("Entering pasta",folder,"at {}".format(datetime.now().strftime("%H:%M:%S")))
    for frame in Input["Frame"].unique():
        Predict = Input.loc[Input["Frame"]==frame]
        file = open("./{}/bypassdecision_frame_{}.txt".format(folder,frame),"w")
        for i in range(0,Predict.shape[0]):
            if ((Predict.iloc[i]["Height"]+64)<Dados.height):
                Input_Depth0 = map_CU(Predict.iloc[i],Dados.height,Dados.width,64,Video,Dados.QP)
                result = Depth_0.predict(Input_Depth0)
                string = str(result[0].argmax())
            else:
                string = "1"
            if string == "1" :
                fun = generate_Depth(Predict.iloc[i],32)
                for j in range(0,fun.shape[0]):
                    if ((fun.iloc[j]["Height"]+32)<Dados.height):
                        Input_Depth1 = map_CU(fun.iloc[j],Dados.height,Dados.width,32,Video,Dados.QP)
                        result = Depth_1.predict(Input_Depth1)
                        check = str(result[0].argmax())
                        string = string + " " + check
                    else:
                        string = string + " 1"
                        check = "1"
                    if check == "1":
                        not_fun = generate_Depth(fun.iloc[j],16)
                        for k in range(0,not_fun.shape[0]):
                            if ((not_fun.iloc[k]["Height"]+16)<Dados.height):
                                Input_Depth2 = map_CU(not_fun.iloc[k],Dados.height,Dados.width,16,Video,Dados.QP)
                                result = Depth_2.predict(Input_Depth2)
                                string_2 = string_2 + " " + str(result[0].argmax())
                            else:
                                string_2 = string_2 + " 1"
            string = string + string_2
            string_2 = ""
            file.write("CU ({:4d},{:4d}) - {}\n".format(Predict.iloc[i]["Height"],Predict.iloc[i]["Width"],string))
        file.close()
    del(Input)
    del(Video)
    del(Dados)
    sai = datetime.now()
    diferenca = (sai-entra).total_seconds()/60
    print("\n Tempo para executar na pasta = {:.2f} MINUTOS".format(diferenca))
    print("\nBypass com DeepLearning Criado na pasta {}, saindo da pasta as {}\n\n".format(folder,datetime.now().strftime("%H:%M:%S")))

Entering pasta Kimon_22 at 00:15:59

 Tempo para executar na pasta = 22.17 MINUTOS

Bypass com DeepLearning Criado na pasta Kimon_22, saindo da pasta as 00:38:10


Entering pasta Kimon_27 at 00:38:10

 Tempo para executar na pasta = 16.15 MINUTOS

Bypass com DeepLearning Criado na pasta Kimon_27, saindo da pasta as 00:54:19


Entering pasta Kimon_32 at 00:54:20

 Tempo para executar na pasta = 14.15 MINUTOS

Bypass com DeepLearning Criado na pasta Kimon_32, saindo da pasta as 01:08:29


Entering pasta Kimon_37 at 01:08:30

 Tempo para executar na pasta = 13.71 MINUTOS

Bypass com DeepLearning Criado na pasta Kimon_37, saindo da pasta as 01:22:12


Entering pasta ParkS_22 at 01:22:13

 Tempo para executar na pasta = 35.97 MINUTOS

Bypass com DeepLearning Criado na pasta ParkS_22, saindo da pasta as 01:58:11


Entering pasta ParkS_27 at 01:58:12

 Tempo para executar na pasta = 20.14 MINUTOS

Bypass com DeepLearning Criado na pasta ParkS_27, saindo da pasta as 02:18:20


Entering pasta P